In [7]:
# Author: Saeed Zahran
# Creating epochs and generating evoked responses

import pathlib
import matplotlib

import mne
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
from pathlib import Path

import numpy as np
import scipy.ndimage

In [8]:
# Load the two raw data files
raw1 = mne.io.read_raw_fif('GS_01_analysis_01-raw.fif', preload=True)
raw2 = mne.io.read_raw_fif('GS_02_analysis_01-raw.fif', preload=True)
raw = mne.io.concatenate_raws([raw1, raw2])

Opening raw data file GS_01_analysis_01-raw.fif...
    Range : 0 ... 119999 =      0.000 ...   119.999 secs
Ready.
Reading 0 ... 119999  =      0.000 ...   119.999 secs...
Opening raw data file GS_02_analysis_01-raw.fif...
    Range : 0 ... 541999 =      0.000 ...   541.999 secs
Ready.
Reading 0 ... 541999  =      0.000 ...   541.999 secs...


In [9]:
raw.filter(l_freq=0.1, h_freq=40)

Filtering raw data in 2 contiguous segments
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 33001 samples (33.001 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    4.1s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    8.9s


Measurement date,"March 26, 2024 09:03:13 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,2350 points
Good channels,"207 Magnetometers, 17 Reference Magnetometers, 32 misc, 1 Stimulus"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.10 Hz
Lowpass,40.00 Hz


In [12]:
events = mne.find_events(raw, stim_channel="STI 014")
event_id = {
    "eyes_closed": 1,
    "eyes_open": 2,
}

2 events found on stim channel STI 014
Event IDs: [1 2]


In [13]:
# Define the duration of each epoch (in seconds)
epoch_duration = 5

# Create fixed-length epochs for the entire concatenated raw data
epochs = mne.make_fixed_length_epochs(raw, duration=epoch_duration, preload=True)



# Apply baseline correction
epochs.apply_baseline((0, 0))

epochs.plot()

Not setting metadata
132 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 132 events and 5000 original time points ...
0 bad epochs dropped
Applying baseline correction (mode: mean)


In [14]:
epochs.plot_image()

Not setting metadata
132 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
132 matching events found
No baseline correction applied
0 projection items activated
combining channels using RMS (mag channels)
combining channels using "gfp"
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


[<Figure size 640x480 with 3 Axes>, <Figure size 640x480 with 3 Axes>]

In [15]:
report = mne.Report(title="Epochs")
report.add_epochs(epochs=epochs, title='Epochs from "epochs"')
report.save("report_epochs.html", overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
    Using multitaper spectrum estimation with 7 DPSS windows
Plotting power spectral density (dB=True).
Averaging across epochs...


C:\Users\User\AppData\Local\Temp\ipykernel_17040\743346917.py:2: UserWarning: Zero value in spectrum for channels MEG 053, MEG 067, MEG 102, MEG 137, MEG 154, MEG 181, MEG 182, MEG 183
  report.add_epochs(epochs=epochs, title='Epochs from "epochs"')
C:\Users\User\AppData\Local\Temp\ipykernel_17040\743346917.py:2: RuntimeWarning: Estimated head radius (11.2 cm) is above the 99th percentile for adult head size.
  report.add_epochs(epochs=epochs, title='Epochs from "epochs"')
C:\Users\User\AppData\Local\Temp\ipykernel_17040\743346917.py:2: UserWarning: Infinite value in PSD for channels MEG 053, MEG 067, MEG 102, MEG 137, MEG 154, MEG 181, MEG 182, MEG 183.
These channels might be dead.
  report.add_epochs(epochs=epochs, title='Epochs from "epochs"')


Saving report to : C:\Users\User\Documents\NYU\MEG-EEG-pipeline\report_epochs.html


'C:\\Users\\User\\Documents\\NYU\\MEG-EEG-pipeline\\report_epochs.html'

In [17]:
# Saving epochs
epochs.save(pathlib.Path('out_data') / 'epochs_epo.fif',
            overwrite=True)

Overwriting existing file.
Overwriting existing file.


In [18]:
# Extract epochs for "eyes closed" and "eyes open" events using event IDs
epochs_closed = epochs[event_id["eyes_closed"]]
epochs_open = epochs[event_id["eyes_open"]]

In [19]:
# Creating evoked data
evoked_closed = epochs_closed.average()
evoked_open = epochs_open.average()

evoked_closed.plot(spatial_colors=True)

C:\Users\User\AppData\Local\Temp\ipykernel_17040\3385494830.py:5: RuntimeWarning: Estimated head radius (11.2 cm) is above the 99th percentile for adult head size.
  evoked_closed.plot(spatial_colors=True)


<Figure size 640x300 with 2 Axes>

In [20]:
# Saving evoked data
mne.write_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                  evoked=[evoked_closed, evoked_open])

FileExistsError: Destination file exists. Please use option "overwrite=True" to force overwriting.

In [21]:
# Reading evoked data
evokeds = mne.read_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif')

Reading C:\Users\User\Documents\NYU\MEG-EEG-pipeline\out_data\evokeds_ave.fif ...
    Found the data of interest:
        t =       0.00 ...    4999.00 ms (1)
        0 CTF compensation matrices available
        nave = 1 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [0, 0] s)
    Found the data of interest:
        t =       0.00 ...    4999.00 ms (1)
        0 CTF compensation matrices available
        nave = 1 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [0, 0] s)


In [22]:
report = mne.Report(title="Evoked")
report.add_evokeds(
    evokeds=evokeds,
    n_time_points=5,
)
report.save("report_evoked.html", overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css


C:\Users\User\anaconda3\envs\mne\Lib\site-packages\joblib\parallel.py:1847: RuntimeWarning: Estimated head radius (11.2 cm) is above the 99th percentile for adult head size.
  res = func(*args, **kwargs)
C:\Users\User\anaconda3\envs\mne\Lib\site-packages\joblib\parallel.py:1847: RuntimeWarning: Estimated head radius (11.2 cm) is above the 99th percentile for adult head size.
  res = func(*args, **kwargs)
C:\Users\User\anaconda3\envs\mne\Lib\site-packages\joblib\parallel.py:1847: RuntimeWarning: Estimated head radius (11.2 cm) is above the 99th percentile for adult head size.
  res = func(*args, **kwargs)
C:\Users\User\anaconda3\envs\mne\Lib\site-packages\joblib\parallel.py:1847: RuntimeWarning: Estimated head radius (11.2 cm) is above the 99th percentile for adult head size.
  res = func(*args, **kwargs)
C:\Users\User\anaconda3\envs\mne\Lib\site-packages\joblib\parallel.py:1847: RuntimeWarning: Estimated head radius (11.2 cm) is above the 99th percentile for adult head size.
  res = fu

Saving report to : C:\Users\User\Documents\NYU\MEG-EEG-pipeline\report_evoked.html


'C:\\Users\\User\\Documents\\NYU\\MEG-EEG-pipeline\\report_evoked.html'